# RAG 테스트

In [2]:
%pip install --quiet --disable-pip-version-check -r ../../requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [31]:
%pip install pdfplumber

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 3.7 MB/s eta 0:00:00 MB/s eta 0:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 3.8 MB/s eta 0:00:003.9 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [pdfplumber]━━━━━━━ 2/3 [pdfplumber]
Note: you may need to restart the kernel to use updated packages.


## 1. 환경 변수 로드

In [1]:
# 환경 변수 로드
from dotenv import load_dotenv

load_dotenv()

True

## 2. 문서 스캔

In [2]:
import os
# 문서가 있는 디렉토리 지정
directory = "./docs"

all_files = []
docs = []

for root, dirs, files in os.walk(directory):
  for file in files:
    full_path = os.path.join(root, file)
    all_files.append(full_path)

print(all_files)

['./docs/testmd.md', './docs/testtest.md', './docs/fakemd.txt', './docs/pdf/AI Trend with LLM.pdf', './docs/md/testmdmd.md']


In [3]:
# 파일 확장자 감지하기
def checkExtension(extention: str, files: str):
  """
  extention: 확장자명 (md, pdf 등) <br/>
  files: 파일 목록 <br/>
  -> str[]
  """
  results = []
  for file in files:
    file_name = file.split('.')
    if (file_name[-1] == extention):
      results.append(file)
  return results

### 사용할 문서가 마크다운일 경우

In [4]:
def get_md_files(files):
  """
  files: 배열로 변환할 파일 목록 <br/>
  -> str[]
  """
  md_files = checkExtension('md', files)

  print("감지된 markdown 자료: ", end="")
  print(md_files)

  # 실제로 파일 읽어서 배열에 집어넣기
  md_docs = []

  for file in md_files:
    with open(file, "r", encoding="utf-8") as f:
      print("파일 읽음: "+file)
      text = f.read()
      text = text.lower()
      title = file.split('/')[-1]
      text = f"# {title}\n" + text
      md_docs.append(text)
  return md_docs

for a in get_md_files(all_files):
  print(a)

감지된 markdown 자료: ['./docs/testmd.md', './docs/testtest.md', './docs/md/testmdmd.md']
파일 읽음: ./docs/testmd.md
파일 읽음: ./docs/testtest.md
파일 읽음: ./docs/md/testmdmd.md
# testmd.md
asdfa
asdfadfadsfa
asdfasfdfasfdsfass

# ㅁㄴㅇㄹㅁㄴㅇㄹㄴㄹ

# testtest.md
asdfasdsfsfsafdsf

# testmdmd.md



### 사용할 문서가 PDF 일 경우

In [ ]:
import pdfplumber

def get_pdf_files(files):
  pdf_files = checkExtension('pdf', files)

  print("감지된 pdf 자료: ", end="")
  print(pdf_files)

  # 실제로 파일 읽어서 배열에 집어넣기
  pdf_docs = []

  for file in pdf_files:
    text = ""
    with pdfplumber.open(file) as pdf:
      for page in pdf.pages:
        tmp = page.extract_text()
        if tmp.find("(c) 2025. codingiscoffee Co. all rights reserved."):
          tmp = tmp.replace("(c) 2025. codingiscoffee Co. all rights reserved.", "")
        text += tmp + "\n"
    pdf_docs.append(text)
        
  return pdf_docs

for a in get_pdf_files(all_files):
  print(a)

감지된 pdf 자료: ['./docs/pdf/AI Trend with LLM.pdf']
LLM Trend (feat. Llama)
LLM, SLM, RAG, Agent, ... ?
뭐가 너무 많은데
김용담 강사

Contents
1. Llama 3.1 LLM Trend
을 중심으로 살펴보는
2. Llama 3.2 Large Multi-Model
를 통해 보는 의 이해
3. RAG
에 대한 이해
4. LLM App
구축시 고려할 사항들에 대한 이해

1. Llama 3.1 LLM Trend
을 중심으로 살펴보는

Language Model ?
이란 무엇인가
?
컴퓨터가 이해하는 텍스트란
v = (1, 3, 4, ...., ..., )
character encoding (word) embedding
Source: https://velog.io/@goggling/유니코드와-UTF-이해하기
Source: https://towardsdatascience.com/deep-learning-4-embedding-layers-f9a02d55ac12
Language Model ?
이란 무엇인가
= Next Token Prediction
맥락의 이해
going go eating

Language Model ?
이란 무엇인가
Neural Language Modeling
mat
Source : https://lena-voita.github.io/nlp_course/language_modeling.html

Language Model ?
이란 무엇인가
Neural Language Modeling in Auto-Regressive manner
Source : https://lena-voita.github.io/nlp_course/language_modeling.html

LLM
이제는 대 의 시대
LLM !
모든 문제를 으로 해결해보자

Large Language Model
Neural Scaling Law
• GPT-2
모델 이후부터 학습 규모를 키우면 성능이 올라갈거라는

## 3. 임베딩/벡터스토어 적재

In [59]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

docs = get_md_files(all_files) + get_pdf_files(all_files)

# 문서 청크로 나누기
splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
splits = splitter.create_documents(docs)

emb = OpenAIEmbeddings(model="text-embedding-3-large")
vstore = FAISS.from_documents(splits, emb)
retriever = vstore.as_retriever(k=3)

감지된 markdown 자료: ['./docs/testmd.md', './docs/testtest.md', './docs/md/testmdmd.md']
파일 읽음: ./docs/testmd.md
파일 읽음: ./docs/testtest.md
파일 읽음: ./docs/md/testmdmd.md
감지된 pdf 자료: ['./docs/pdf/AI Trend with LLM.pdf']


In [63]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

template = """당신은 문서 기반 QA 비서입니다.
질문: {question}
검색 컨텍스트:
{context}
요청: 위 컨텍스트만 근거로 간결히 한국어로 답하세요.
"""

prompt = ChatPromptTemplate.from_template(template)

def format_docs(docs):
    return "\n\n".join(d.page_content for d in docs)

llm = ChatOpenAI(model="gpt-4o", temperature=0)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
print(prompt)
print(rag_chain.invoke("rag의 순서는?"))

input_variables=['context', 'question'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='당신은 문서 기반 QA 비서입니다.\n질문: {question}\n검색 컨텍스트:\n{context}\n요청: 위 컨텍스트만 근거로 간결히 한국어로 답하세요.\n'), additional_kwargs={})]
RAG의 순서는 다음과 같습니다:

1. Document Loading
2. Text Splitting
3. Embedding
4. Vector Store (= Indexing)
5. Retrieving
6. Generating
